In [ ]:
# pyright: reportMissingImports=false
# pyright: reportMissingModuleSource=false

import uuid
import random
import hashlib
import os
import sys
import time
import logging
import re
import pdb
import datetime
import json
from datetime import datetime, timedelta
import yaml
import pytest
import importlib
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import requests
import pandas_gbq
from sklearn.model_selection import ParameterGrid, ParameterSampler
from scipy.signal import argrelextrema
from dreams_core.googlecloud import GoogleCloud as dgc
from dreams_core import core as dc
import matplotlib.pyplot as plt
import seaborn as sns
import progressbar

# load dotenv
load_dotenv()

# Custom format function for displaying numbers/
pd.set_option('display.float_format', lambda x: f'{x:.12g}')
# pd.reset_option('display.float_format')

# Dark mode charts
plt.rcParams['figure.facecolor'] = '#181818'  # Custom background color (dark gray in this case)
plt.rcParams['axes.facecolor'] = '#181818'
plt.rcParams['text.color'] = '#afc6ba'
plt.rcParams['axes.labelcolor'] = '#afc6ba'
plt.rcParams['xtick.color'] = '#afc6ba'
plt.rcParams['ytick.color'] = '#afc6ba'
plt.rcParams['axes.titlecolor'] = '#afc6ba'

# import local modules
# pyright: reportMissingImports=false
sys.path.append('..//src')
import utils as u
import training_data.data_retrieval as dr
import training_data.profits_row_imputation as pri
import coin_wallet_metrics.coin_wallet_metrics as cwm
import coin_wallet_metrics.indicators as ind
import feature_engineering.feature_generation as fg
import feature_engineering.flattening as flt
import feature_engineering.preprocessing as prp
import modeling as m
import insights.analysis as ia
import insights.model_input_flows as mif


# reload all modules
modules = [dr, pri, fg, flt, prp, cwm, ind, m, ia, mif, u]
[importlib.reload(module) for module in modules]

# load all configs
config, metrics_config, modeling_config, experiments_config = u.load_all_configs('../config')

# configure logger
logger = dc.setup_logger()
logger.setLevel(logging.INFO)


## All windows data prep

In [ ]:
[importlib.reload(module) for module in modules]  # Reload all modules
config, metrics_config, modeling_config, experiments_config = u.load_all_configs('../config')  # Reload all configs

# Retrieve all windows datasets
macro_trends_df, market_data_df, profits_df, prices_df = mif.retrieve_all_windows_training_data(config,
                                                                                            metrics_config)

#### generate windows

In [ ]:
[importlib.reload(module) for module in modules]  # Reload all modules
config, metrics_config, modeling_config, experiments_config = u.load_all_configs('../config')  # Reload all configs

# Generate time_windows config overrides that will modify each window's config settings
time_windows = mif.generate_time_windows(config)

all_flattened_dfs = []
all_flattened_filepaths = []

for n, time_window in enumerate(time_windows):

    # Prepare time window config files
    window_config, window_metrics_config, window_modeling_config = (
        mif.prepare_configs(modeling_config['modeling']['config_folder'], time_window))

    # Generate flattened feature dfs for all datasets for the window
    window_flattened_dfs, window_flattened_filepaths = mif.generate_window_flattened_dfs(
        market_data_df,
        macro_trends_df,
        profits_df,
        prices_df,
        window_config,
        window_metrics_config,
        window_modeling_config
    )

    # Store window's flattened features
    all_flattened_dfs.extend(window_flattened_dfs)
    all_flattened_filepaths.extend(window_flattened_filepaths)

all_flattened_filepaths

## Preprocessing sequence

In [ ]:
all_flattened_filepaths[1]

In [ ]:
[importlib.reload(module) for module in modules]  # Reload all modules
config, metrics_config, modeling_config, experiments_config = u.load_all_configs('../config')  # Reload all configs

# Identify filepath of parent directory
parent_directory = os.path.join(modeling_config['modeling']['modeling_folder'],
                               'outputs/preprocessed_outputs/')

# for filepath in all_flattened_filepaths:
filepath = all_flattened_filepaths[2]

dataset = prp.extract_dataset_key_from_filepath(filepath, parent_directory)



In [ ]:
all_flattened_filepaths

In [ ]:
filepath = all_flattened_filepaths[0]
print(filepath)

if os.path.exists(filepath):
    df = pd.read_csv(filepath)

df.columns

In [ ]:
# Initialize an empty list to hold DataFrames
df_list = []
missing_files = []

# Regex to extract the date pattern %Y-%m-%d_%H-%M from the filename
date_pattern = re.compile(r'\d{4}-\d{2}-\d{2}_\d{2}-\d{2}')

# Loop through the input_file_tuples (filename, fill_strategy)
for filepath in all_flattened_filepaths:

    if os.path.exists(filepath):
        df = pd.read_csv(filepath)

        # Check if coin_id column exists if the fill strategy requires it
        if fill_strategy in ['drop_records','fill_zeros']:
            if 'coin_id' not in df.columns:
                raise ValueError(f"coin_id column is missing in {filename}")

            # Raise an error if there are duplicate coin_ids in the file
            if df['coin_id'].duplicated().any():
                raise ValueError(f"Duplicate coin_ids found in file: {filename}")

        # Extract the date string from the filename
        match = date_pattern.search(filename)
        if not match:
            raise ValueError(f"No valid date string found in the filename: {filename}")
        date_string = match.group()  # e.g., '2024-09-13_14-44'
        metric_string = filename.split(date_string)[0].rstrip('_')

        # Add column suffixes based on the count of metric_string
        if metric_string_count[metric_string] > 1:
            suffix = f"{metric_string}_{date_string}"
        else:
            suffix = metric_string

        # Check if this suffix has been used before and append a numerical suffix if necessary
        for column in df.columns:
            if column != 'coin_id':
                column_with_suffix = f"{column}_{suffix}"

                # Check if suffix exists in the count, increment if necessary
                if column_with_suffix in column_suffix_count:
                    column_suffix_count[column_with_suffix] += 1
                    suffix_count = column_suffix_count[column_with_suffix]
                    column_with_suffix = f"{column_with_suffix}_{suffix_count}"
                else:
                    column_suffix_count[column_with_suffix] = 1

                df = df.rename(columns={column: column_with_suffix})

        # Append DataFrame and fill strategy to the list for processing
        df_list.append((df, fill_strategy, filename))
    else:
        missing_files.append(filename)

# Merge the output DataFrames based on their fill strategies
training_data_df, merge_logs_df = merge_and_fill_training_data(df_list)

# Log the results
logger.debug("%d files were successfully merged into training_data_df.", len(df_list))
if missing_files:
    logger.warning("%d files could not be found: %s",
                    len(missing_files), ', '.join(missing_files))
else:
    logger.debug("All specified files were found and merged into training_data_df.")

return training_data_df, merge_logs_df


In [ ]:

def merge_and_fill_training_data(
    df_list: list[tuple[pd.DataFrame, str, str]]
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Merges a list of DataFrames on 'coin_id' and applies specified fill strategies for missing
    values. Generates a log of the merging process with details on record counts and modifications.

    Params:
    - df_list (list of tuples): Each tuple contains:
        - df (pd.DataFrame): A DataFrame to merge.
        - fill_strategy (str): The strategy to handle missing values ('fill_zeros', 'drop_records').
        - filename (str): The name of the input file, used for logging.

    Returns:
    - training_data_df (pd.DataFrame): Merged DataFrame with applied fill strategies.
    - merge_logs_df (pd.DataFrame): Log DataFrame detailing record counts for each input DataFrame.
    """
    if not df_list:
        raise ValueError("No DataFrames to merge.")

    # Initialize the log DataFrame
    merge_logs = []

    # Pull a unique set of all coin_ids across all DataFrames
    all_coin_ids = set()
    for df, _, _ in df_list:
        if 'coin_id' in df.columns:
            all_coin_ids.update(df['coin_id'].unique())

    # Makes a new df with all coin_ids in a column
    training_data_df = pd.DataFrame(all_coin_ids, columns=['coin_id'])

    # Iterate through df_list and merge each one
    for df, fill_strategy, filename in df_list:

        # if the df is a macro_series without a coin_id, cross join it to all coin_ids
        if fill_strategy == 'extend':
            original_coin_ids = set()
            training_data_df = training_data_df.merge(df, how='cross')

        else:
            # Merge with the full coin_id set (outer join)
            original_coin_ids = set(df['coin_id'].unique())  # Track original coin_ids
            training_data_df = pd.merge(training_data_df, df, on='coin_id', how='outer')

            # Apply the fill strategy
            if fill_strategy == 'fill_zeros':
                # Fill missing values with 0
                training_data_df.fillna(0, inplace=True)
            elif fill_strategy == 'drop_records':
                # Drop rows with missing values for this DataFrame's columns
                training_data_df.dropna(inplace=True)
            else:
                raise ValueError(f"Invalid fill strategy '{fill_strategy}' found in config.yaml.")

        # Calculate log details
        final_coin_ids = set(training_data_df['coin_id'].unique())
        filled_ids = final_coin_ids - original_coin_ids  # Present in final, missing in original

        # Add log information for this DataFrame
        merge_logs.append({
            'file': filename,  # Use filename for logging
            'original_count': len(original_coin_ids),
            'filled_count': len(filled_ids)
        })

    # Ensure no duplicate columns after merging
    if training_data_df.columns.duplicated().any():
        raise ValueError("Duplicate columns found after merging.")

    # Raise an error if there are any null values in the final DataFrame
    if training_data_df.isnull().any().any():
        raise ValueError("Null values detected in the final merged DataFrame.")

    # Convert logs to a DataFrame
    merge_logs_df = pd.DataFrame(merge_logs)

    return training_data_df, merge_logs_df


## Window specific metrics

#### Window-specific metrics

In [ ]:
[importlib.reload(module) for module in modules]  # Reload all modules
config, metrics_config, modeling_config, experiments_config = u.load_all_configs('../config')  # Reload all configs


# Prepare time window config files
config, metrics_config, modeling_config = mif.prepare_configs(modeling_config['modeling']['config_folder'], time_window)


In [ ]:
all_flattened_filepaths

In [ ]:

all_flattened_dfs[3]

In [ ]:
all_flattened_filepaths.extend([flattened_market_data_filepath])

all_flattened_filepaths

#### Cohorts

In [ ]:
[importlib.reload(module) for module in modules]  # Reload all modules
config, metrics_config, modeling_config, experiments_config = u.load_all_configs('../config')  # Reload all configs

def generate_wallet_cohort_features(
        profits_df,
        prices_df,
        config,
        metrics_config,
        modeling_config
    ):
    """
    Generates a window-specific feature set from the full all windows dataset. The window-specific
    features are saved as a csv and returned, along with the csv filepath.

    This function differs from the time_series set because it only flattens on date, since this
    dataset doesn't have coin_id.

    Params:
    - all_windows_time_series_df (DataFrame): df containing all metrics and indicators for a time
        series dataset.
    - config: config.yaml that has the dates for the specific time window
    - metrics_config: metrics_config.yaml
    - modeling_config: modeling_config.yaml

    Returns:
    - flattened_metrics_df (DataFrame): the flattened version of the original df, with columns for
        the configured aggregations and rolling metrics for all value columns and indicators.
    - flattened_metrics_filepath (string): the filepath to where the flattened_metrics_df is saved
    """

    # 1. Impute all required dates
    # ----------------------------
    # Identify all required imputation dates
    imputation_dates = pri.identify_imputation_dates(config)

    # Impute all required dates
    window_profits_df = pri.impute_profits_for_multiple_dates(profits_df, prices_df, imputation_dates, n_threads=24)
    window_profits_df = (window_profits_df[(window_profits_df['date'] >= pd.to_datetime(min(imputation_dates))) &
                                        (window_profits_df['date'] <= pd.to_datetime(max(imputation_dates)))])


    # 2. Generate metrics and indicators for all cohorts
    # --------------------------------------------------
    for cohort_name in metrics_config['wallet_cohorts']:

        # load configs
        dataset_metrics_config = metrics_config['wallet_cohorts'][cohort_name]
        dataset_config = config['datasets']['wallet_cohorts'][cohort_name]

        # identify wallets in the cohort based on the full lookback period
        cohort_summary_df = cwm.classify_wallet_cohort(window_profits_df, dataset_config, cohort_name)
        cohort_wallets = cohort_summary_df[cohort_summary_df['in_cohort']]['wallet_address']

        # If no cohort members were identified, continue
        if len(cohort_wallets) == 0:
            logger.info("No wallets identified as members of cohort '%s'", cohort_name)
            continue

        # Generate cohort buysell_metrics
        cohort_metrics_df = cwm.generate_buysell_metrics_df(profits_df,
                                                            config['training_data']['training_period_end'],
                                                            cohort_wallets)

        # Generate cohort indicator metrics
        cohort_metrics_df = ind.generate_time_series_indicators(cohort_metrics_df,
                                                                metrics_config['wallet_cohorts'][cohort_name],
                                                                'coin_id')

        # Flatten cohort metrics
        flattened_cohort_df, flattened_cohort_filepath = fe.generate_window_time_series_features(
            cohort_metrics_df,
            config,
            dataset_metrics_config,
            modeling_config
        )

In [ ]:
[importlib.reload(module) for module in modules]  # Reload all modules
config, metrics_config, modeling_config, experiments_config = u.load_all_configs('../config')  # Reload all configs




In [ ]:
# Flatten the metrics DataFrame to be keyed only on coin_id
flattened_metrics_df = fe.flatten_coin_date_df(
    cohort_metrics_df,
    metrics_config['wallet_cohorts'][cohort_name],
    config['training_data']['training_period_end']  # Ensure data is up to training period end
)
flattened_metrics_df.columns

In [ ]:
# Add time window modeling period start
flattened_metrics_df.loc[:,'time_window'] = config['training_data']['modeling_period_start']

# Save the flattened output and retrieve the file path
_, flattened_metrics_filepath = fe.save_flattened_outputs(
    flattened_metrics_df,
    os.path.join(
        modeling_config['modeling']['modeling_folder'],  # Folder to store flattened outputs
        'outputs/flattened_outputs'
    ),
    'market_data',  # Descriptive metadata for the dataset
    config['training_data']['modeling_period_start']  # Ensure data starts from modeling period
)

In [ ]:
# Generate cohort buysell_metrics
cohort_metrics_df = cwm.generate_buysell_metrics_df(profits_df,
                                                    config['training_data']['training_period_end'],
                                                    cohort_wallets)


cohort_metrics_df.shape

In [ ]:

# Generate cohort indicator metrics
cohort_metrics_df = ind.generate_time_series_indicators(cohort_metrics_df,
                                                        metrics_config['wallet_cohorts'][cohort_name],
                                                        'coin_id')

cohort_metrics_df.shape

In [ ]:
cohort_metrics_df.describe()

In [ ]:
time_series_df = cohort_metrics_df
start_date = config['training_data']['training_period_start']
end_date = config['training_data']['training_period_end']
id_column='coin_id'
drop_outside_date_range=True

# Convert params to datetime
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

# Create copy of df
time_series_df = time_series_df.copy()

# Drop all rows with any NaN values
time_series_df = time_series_df.dropna()

# Define a function to check if a date range has full coverage
def has_full_coverage(min_date, max_date):
    return (min_date <= start_date) and (max_date >= end_date)

if id_column:
    # Multi-series data
    series_data_range = time_series_df.groupby(id_column, observed=True)['date'].agg(['min', 'max'])
    full_duration_series = series_data_range[series_data_range.apply(lambda x: has_full_coverage(x['min'], x['max']), axis=1)].index
else:
    # Single-series data
    series_data_range = time_series_df['date'].agg(['min', 'max'])
    full_duration_series = [0] if has_full_coverage(series_data_range['min'], series_data_range['max']) else []

# Calculate coverage statistics
full_coverage_count = len(full_duration_series)

# # Split the dataframe
# if id_column:
#     # Convert id column to categorical to reduce memory usage
#     time_series_df[id_column] = time_series_df[id_column].astype('category')
#     full_coverage_df = time_series_df[time_series_df[id_column].isin(full_duration_series)]
#     partial_coverage_df = time_series_df[~time_series_df[id_column].isin(full_duration_series)]
# else:
#     full_coverage_df = time_series_df if full_coverage_count else pd.DataFrame(columns=time_series_df.columns)
#     partial_coverage_df = time_series_df if not full_coverage_count else pd.DataFrame(columns=time_series_df.columns)

# logger.info("Split df with dimensions %s into %s full coverage records and %s partial coverage records.",
#             time_series_df.shape,
#             len(full_coverage_df),
#             len(partial_coverage_df))

# if drop_outside_date_range:
#     # Remove rows outside the date range for both dataframes
#     full_coverage_df = (full_coverage_df[(full_coverage_df['date'] >= start_date) &
#                                             (full_coverage_df['date'] <= end_date)])
#     partial_coverage_df = (partial_coverage_df[(partial_coverage_df['date'] >= start_date) &
#                                                 (partial_coverage_df['date'] <= end_date)])

#     # Log the number of remaining records
#     total_remaining = len(full_coverage_df) + len(partial_coverage_df)
#     logger.info("After removing records outside the date range, %s records remain.",
#                 total_remaining)

# # return full_coverage_df, partial_coverage_df

time_series_df.shape
full_coverage_count

In [ ]:
end_date

In [ ]:
series_data_range = time_series_df.groupby(id_column, observed=True)['date'].agg(['min', 'max'])
series_data_range

In [ ]:


cohort_metrics_df2.shape

In [ ]:
cohort_metrics_df2

In [ ]:

# If no cohort members were identified, continue
if len(cohort_wallets) == 0:
    logger.info("No wallets identified as members of cohort '%s'", cohort_name)
    continue

# Generate cohort buysell_metrics
cohort_metrics_df = cwm.generate_buysell_metrics_df(profits_df,
                                                    config['training_data']['training_period_end'],
                                                    cohort_wallets)

# Generate cohort indicator metrics
cohort_metrics_df = ind.generate_time_series_indicators(cohort_metrics_df,
                                                        metrics_config['wallet_cohorts'][cohort_name],
                                                        'coin_id')

# Split df to only training_period
cohort_metrics_df,_ = cwm.split_dataframe_by_coverage(cohort_metrics_df,
                                                    config['training_data']['training_period_start'],
                                                    config['training_data']['modeling_period_end'],
                                                    id_column='coin_id',
                                                    drop_outside_date_range=True)

cohort_metrics_df.shape

In [ ]:

# # Add time window modeling period start
# cohort_metrics_df.loc[:,'time_window'] = config['training_data']['modeling_period_start']

cohort_metrics_df.columns

In [ ]:
cohort_metrics_df.shape

In [ ]:
# Market data: generate window-specific flattened metrics
flattened_market_data_df, flattened_market_data_filepath = fe.generate_window_time_series_features(
    market_data_df,
    config,
    metrics_config,
    modeling_config
)

In [ ]:
dataset_metrics_config['total_bought']

In [ ]:
cohort_metrics_df.columns

In [ ]:
metrics_config['wallet_cohorts']['whales']

In [ ]:
u.df_mem(profits_df)

In [ ]:
"""
IN WINDOW FUNCTIONS

market_data_df: just filter to window
macro_trends_df: just filter to window

profits_df
1. identify all dates needed
    all cohort lookback window starts
    training_period_start
    training_period_end
    modeling_period_start
    modeling_period_end
2. impute them
3. filter df to only dates between earliest and latest dates


4. wallet cohorts and buysell metrics
5. indicators
6. filter to window
"""

# def build_time_window_model_input(n, window, config, metrics_config, modeling_config):
#     """
#     Generates training data for each of the config.training_data.additional_windows.

#     Params:
#         n (int): The lookback number of the time window (e.g 0,1,2)
#         window (Dict): The config override dict with the window's modeling_period_start
#         config: config.yaml
#         metrics_config: metrics_config.yaml
#         modeling_config: modeling_config.yaml

#     Returns:
#         model_data (Dict): Dictionary containing all of the modeling features and variables:
#             X_train, X_test (DataFrame): Model training features
#             y_train, y_test (pd.Series): Model target variables
#             returns_test (DataFrame): The actual returns of each coin_id in each time_window.
#                 - coin_id: Index (str)
#                 - time_window: Index (int)
#                 - returns: value column (float)
#     """

# Prepare the full configuration by applying overrides from the current trial config
config, metrics_config, modeling_config = prepare_configs(modeling_config['modeling']['config_folder'], window)

# Define window start and end dates
start_date = config['training_data']['training_period_start']
end_date = config['training_data']['modeling_period_end']

# Rebuild market data
market_data_df = dr.retrieve_market_data()
market_data_df, _ = cwm.split_dataframe_by_coverage(market_data_df, start_date, end_date, id_column='coin_id')
prices_df = market_data_df[['coin_id','date','price']].copy()

# Retrieve macro trends data
macro_trends_df = dr.retrieve_macro_trends_data()
macro_trends_df = cwm.generate_macro_trends_features(macro_trends_df, config)

# Rebuild profits_df
if 'profits_df' not in locals():
    profits_df = None
profits_df = rebuild_profits_df_if_necessary(config, prices_df, profits_df)

# Build the configured model input data for the nth window
X_train, X_test, y_train, y_test, returns_test = build_configured_model_input(
                                    profits_df,
                                    market_data_df,
                                    macro_trends_df,
                                    config,
                                    metrics_config,
                                    modeling_config)

# Add time window indices to dfs with coin_ids
X_train['time_window'] = n
X_train.set_index('time_window', append=True, inplace=True)
X_test['time_window'] = n
X_test.set_index('time_window', append=True, inplace=True)
returns_test['time_window'] = n
returns_test.set_index('time_window', append=True, inplace=True)

model_data = {
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test,
    'returns_test': returns_test
}

# return model_data


## Modeling Sequence

In [ ]:


# Generate time_windows config overrides that will modify each window's config settings
time_windows = mif.generate_time_windows(config)

# Initialize empty lists to hold concatenated data
X_train_list, X_test_list = [], []
y_train_list, y_test_list = [], []
returns_test_list = []

for n, window in enumerate(time_windows):

    model_data = mif.build_time_window_model_input(n, window, config, metrics_config, modeling_config)

    # Append the current window's data to the lists
    X_train_list.append(model_data['X_train'])
    X_test_list.append(model_data['X_test'])
    y_train_list.append(model_data['y_train'])
    y_test_list.append(model_data['y_test'])
    returns_test_list.append(model_data['returns_test'])


# Concatenate all the data for each part
X_train = pd.concat(X_train_list, axis=0)
X_test = pd.concat(X_test_list, axis=0)
y_train = pd.concat(y_train_list, axis=0)
y_test = pd.concat(y_test_list, axis=0)
returns_test = pd.concat(returns_test_list, axis=0)

In [ ]:
[importlib.reload(module) for module in modules]  # Reload all modules
config, metrics_config, modeling_config, experiments_config = u.load_all_configs('../config')  # Reload all configs
logger.setLevel(logging.INFO)


# 3.4 Train the model using the current configuration and log the results
model, model_id = m.train_model(
                    X_train,
                    y_train,
                    modeling_config)

# 3.5 Evaluate and save the model performance on the test set to a CSV
metrics_dict, y_pred, y_pred_prob = m.evaluate_model(model, X_test, y_test, model_id, returns_test, modeling_config)

metrics_dict

In [ ]:
feature_importances = model.feature_importances_
features = X_train.columns  # Feature names

# Create a DataFrame with feature names and importance
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': feature_importances
})

# Sort by importance in descending order
importance_df = importance_df.sort_values(by='Importance', ascending=False)
importance_df.head(20)

In [ ]:
importance_df

In [ ]:

for module in modules:
    importlib.reload(module)


# Select y_pred_prob from the classifier, or y_pred from a regressor
predictions = y_pred_prob or y_pred
returns = returns_test['returns']
winsorization_cutoff = modeling_config["evaluation"]["winsorization_cutoff"]


ia.generate_profitability_curves(predictions, returns, winsorization_cutoff)

## Time Window Sequencing

In [ ]:
market_data_df.shape

### checking profits_df downcasts

In [ ]:
start_date = config['training_data']['earliest_window_start']
end_date = config['training_data']['modeling_period_end']
minimum_wallet_inflows = config['data_cleaning']['minimum_wallet_inflows']

# SQL query to retrieve profits data
query_sql = f"""
    -- STEP 1: retrieve profits data and apply USD inflows filter
    -------------------------------------------------------------
    with profits_base as (
        select coin_id
        ,date
        ,wallet_address
        ,profits_cumulative
        ,usd_balance
        ,usd_net_transfers
        ,usd_inflows
        ,usd_inflows_cumulative
        from core.coin_wallet_profits
        where date <= '{end_date}'
    ),

    usd_inflows_filter as (
        select coin_id
        ,wallet_address
        ,max(usd_inflows_cumulative) as total_usd_inflows
        from profits_base
        -- we don't need to include coin-wallet pairs that have no transactions between
        -- the start and end dates
        group by 1,2
    ),

    profits_base_filtered as (
        select pb.*
        from profits_base pb
        join usd_inflows_filter f on f.coin_id = pb.coin_id
            and f.wallet_address = pb.wallet_address
        where f.total_usd_inflows >= {minimum_wallet_inflows}
    ),


    -- STEP 2: create new records for all coin-wallet pairs as of the training_period_start
    ---------------------------------------------------------------------------------------
    -- compute the starting profits and balances as of the training_period_start
    training_start_existing_rows as (
        -- identify coin-wallet pairs that already have a balance as of the period end
        select *
        from profits_base_filtered
        where date = '{start_date}'
    ),
    training_start_needs_rows as (
        -- for coin-wallet pairs that don't have existing records, identify the row closest to the period end date
        select t.*
        ,cmd_previous.price as price_previous
        ,cmd_training.price as price_current
        ,row_number() over (partition by t.coin_id,t.wallet_address order by t.date desc) as rn
        from profits_base_filtered t
        left join training_start_existing_rows e on e.coin_id = t.coin_id
            and e.wallet_address = t.wallet_address

        -- obtain the last price used to compute the balance and profits data
        join core.coin_market_data cmd_previous on cmd_previous.coin_id = t.coin_id and cmd_previous.date = t.date

        -- obtain the training_period_start price so we can update the calculations
        join core.coin_market_data cmd_training on cmd_training.coin_id = t.coin_id and cmd_training.date = '{start_date}'
        where t.date < '{start_date}'
        and e.coin_id is null
    ),
    training_start_new_rows as (
        -- create a new row for the period end date by carrying the balance from the closest existing record
        select t.coin_id
        ,cast('{start_date}' as datetime) as date
        ,t.wallet_address
        -- profits_cumulative is the previous profits_cumulative + the change in profits up to the start_date
        ,((t.price_current / t.price_previous) - 1) * t.usd_balance + t.profits_cumulative as profits_cumulative
        -- usd_balance is previous balance * (1 + % change in price)
        ,(t.price_current / t.price_previous) * t.usd_balance as usd_balance
        -- there were no transfers
        ,0 as usd_net_transfers
        -- there were no inflows
        ,0 as usd_inflows
        -- no change since there were no inflows
        ,usd_inflows_cumulative as usd_inflows_cumulative

        from training_start_needs_rows t
        where rn=1

    ),

    -- STEP 3: merge all records together
    -------------------------------------
    profits_merged as (
        select * from profits_base_filtered
        -- transfers prior to the training period are summarized in training_start_new_rows
        where date >= '{start_date}'

        union all

        select * from training_start_new_rows
    )

    select coin_id
    ,date

    -- replace the memory-intensive address strings with integers
    ,DENSE_RANK() OVER (ORDER BY wallet_address) as wallet_address

    ,profits_cumulative
    ,usd_balance
    ,usd_net_transfers
    ,usd_inflows
    -- set a floor of $0.01 to avoid divide by 0 errors caused by rounding
    ,greatest(0.01,usd_inflows_cumulative) as usd_inflows_cumulative
    from profits_merged
"""

# Run the SQL query using dgc's run_sql method
profits_df = dgc().run_sql(query_sql)

logger.info('Converting columns to memory-optimized formats...')

# Convert coin_id to categorical and date to date
profits_df['coin_id'] = profits_df['coin_id'].astype('category')
profits_df['date'] = pd.to_datetime(profits_df['date'])

# Add total_return column
profits_df['total_return'] = (profits_df['profits_cumulative']
                                / profits_df['usd_inflows_cumulative'])



In [ ]:
profits_df = safe_downcast(profits_df, 'wallet_address', 'int32')

In [ ]:
market_data_df.head()

In [ ]:
np.can_cast(market_data_df['market_cap'].dtype, 'int32', casting='safe')


In [ ]:
market_data_df = safe_downcast(market_data_df, 'volume', 'int32')

### Market Data resequencing

In [ ]:
[importlib.reload(module) for module in modules]  # Reload all modules
config, metrics_config, modeling_config, experiments_config = u.load_all_configs('../config')  # Reload all configs


# Generate time_windows config overrides that will modify each window's config settings
time_windows = mif.generate_time_windows(config)
n = 0
window = time_windows[n]

# Prepare the full configuration by applying overrides from the current trial config
config, metrics_config, modeling_config = mif.prepare_configs(modeling_config['modeling']['config_folder'], window)

# Generate time_windows config overrides that will modify each window's config settings
time_windows = mif.generate_time_windows(config)
n = 0
window = time_windows[n]


# market_data_df, _ = cwm.split_dataframe_by_coverage(market_data_df, start_date, end_date, id_column='coin_id')
# prices_df = market_data_df[['coin_id','date','price']].copy()
market_data_df_full = market_data_df.copy()
market_data_df.shape

In [ ]:
market_data_df_full.head()

In [ ]:
[importlib.reload(module) for module in modules]  # Reload all modules
config, metrics_config, modeling_config, experiments_config = u.load_all_configs('../config')  # Reload all configs


market_data_df = market_data_df_full.copy()
print(market_data_df.columns)
market_data_df = ind.generate_time_series_indicators('market_data', market_data_df, metrics_config)
print(market_data_df.columns)


In [ ]:
isinstance(time_series_df.index, pd.RangeIndex)

In [ ]:
market_data_df.tail()

In [ ]:
[importlib.reload(module) for module in modules]  # Reload all modules
config, metrics_config, modeling_config, experiments_config = u.load_all_configs('../config')  # Reload all configs




market_data_df = market_data_df_full.copy()
value_column = 'price'
value_column_indicators_config = metrics_config['time_series']['market_data'][value_column]['indicators']
id_column = 'coin_id'
market_data_df = ind.generate_column_time_series_indicators(
    market_data_df,
    value_column,
    value_column_indicators_config,
    id_column
)

market_data_df.columns

In [ ]:
id_column = None
if not id_column:
    print('x')

In [ ]:
time_series_df = time_series_df.reset_index()

In [ ]:
[importlib.reload(module) for module in modules]  # Reload all modules
config, metrics_config, modeling_config, experiments_config = u.load_all_configs('../config')  # Reload all configs


# time_series_df = market_data_df[['date','coin_id','price']].copy()
time_series_df = market_data_df_full.copy()
config = config
value_column_indicators_config = metrics_config['time_series']['market_data']['price']['indicators']
value_column = 'price'
id_column='coin_id'

time_series_df = time_series_df.set_index(['coin_id','date'])

# Data Quality Checks and Formatting
if value_column not in time_series_df.columns:
    raise KeyError(f"Input DataFrame does not include column '{value_column}'.")

if time_series_df[value_column].isnull().any():
    raise ValueError(f"The '{value_column}' column contains null values, which are not allowed.")

# Indicator Calculations
# ----------------------
# If there is an id_column, group on it
if id_column:
    groupby_column = id_column
# If there isn't, create a dummy_column for grouping and remove it later
else:
    time_series_df['dummy_group'] = 1
    groupby_column = 'dummy_group'

# For each indicator, loop through all options and add the appropriate column
for indicator, indicator_config in value_column_indicators_config.items():
    if indicator == 'sma':
        windows = indicator_config['parameters']['window']
        for w in windows:
            ind_series = time_series_df.groupby(level=groupby_column, observed=True)[value_column].transform(
                lambda x: ind.calculate_sma(x, w))
            time_series_df[f"{value_column}_{indicator}_{w}"] = ind_series

    elif indicator == 'ema':
        windows = indicator_config['parameters']['window']
        for w in windows:
            ind_series = time_series_df.groupby(level=groupby_column, observed=True)[value_column].transform(
                lambda x: ind.calculate_ema(x, w))
            time_series_df[f"{value_column}_{indicator}_{w}"] = ind_series

    # elif indicator == 'rsi':
    #     windows = indicator_config['parameters']['window']
    #     for w in windows:
    #         ind_series = time_series_df.groupby(level=groupby_column, observed=True)['price'].transform(
    #             lambda x: calculate_rsi(x, w))
    #         time_series_df[f"{value_column}_{indicator}_{w}"] = ind_series

    # elif indicator == 'bollinger_bands_upper':
    #     windows = indicator_config['parameters']['window']
    #     num_std = indicator_config['parameters'].get('num_std', None)
    #     for w in windows:
    #         ind_series = time_series_df.groupby(level=groupby_column, observed=True)['price'].transform(
    #             lambda x: calculate_bollinger_bands(x, 'upper', w, num_std))
    #         time_series_df[f"{value_column}_{indicator}_{w}"] = ind_series

    # elif indicator == 'bollinger_bands_lower':
    #     windows = indicator_config['parameters']['window']
    #     num_std = indicator_config['parameters'].get('num_std', None)
    #     for w in windows:
    #         ind_series = time_series_df.groupby(level=groupby_column, observed=True)['price'].transform(
    #             lambda x: calculate_bollinger_bands(x, 'lower', w, num_std))
    #         time_series_df[f"{value_column}_{indicator}_{w}"] = ind_series

# Remove the dummy column if it was created
if groupby_column == 'dummy_group':
    time_series_df = time_series_df.drop('dummy_group', axis=1)

logger.info("Generated indicators for column '%s' :%s",
            value_column,
            list(value_column_indicators_config.keys()))


In [ ]:
list(value_column_indicators_config.keys())

In [ ]:
logger.info("%s",value_column_indicators_config.keys())

In [ ]:
market_data_df.xs('9d6619f4-b44b-4ff4-9f68-1f563f57e060',level='coin_id').tail()

In [ ]:
market_data_df.sample(15)

In [ ]:
x = indicator_config['parameters'].get('num_std', None)
x

In [ ]:
market_data_df.groupby(level='coin_id', observed=True)['price'].transform(

### indicators implementation

In [ ]:
[importlib.reload(module) for module in modules]  # Reload all modules
config, metrics_config, modeling_config, experiments_config = u.load_all_configs('../config')  # Reload all configs


df = market_data_df.copy()
df = market_data_df.set_index(['coin_id','date'])


# Add Relative Strength Index (RSI)
df['rsi'] = df.groupby(level='coin_id', observed=True)['price'].transform(
    lambda x: ind.calculate_rsi(x, 14))
# Add Money Flow Index (MFI)
df = ind.add_mfi_column(df)

# Calculate MACD with EMAs
df['ema_12'] = df.groupby(level='coin_id', observed=True)['price'].transform(lambda x: ind.calculate_ema(x, 12))
df['ema_26'] = df.groupby(level='coin_id', observed=True)['price'].transform(lambda x: ind.calculate_ema(x, 26))
df = ind.add_crossover_column(df, 'ema_12', 'ema_26', drop_col1=True, drop_col2=True)

# Add Bollinger Bands
df = ind.add_bollinger_bands(df, include_middle=False)
# Add crossover for price and upper band
df = ind.add_crossover_column(df, 'price', 'bollinger_band_upper', drop_col1=False, drop_col2=True)
# Add crossover for price and lower band
df = ind.add_crossover_column(df, 'price', 'bollinger_band_lower', drop_col1=False, drop_col2=True)

# Calculate OBV
df['obv_price_volume'] = ind.generalized_obv(df['price'],df['volume'])


df.head()

## Junkyard

In [ ]:
[importlib.reload(module) for module in modules]  # Reload all modules
config, metrics_config, modeling_config, experiments_config = u.load_all_configs('../config')  # Reload all configs

# Define a function to calculate MFI within each group, similar to the crossovers function
def apply_mfi(group):
    # Reset index to avoid issues with the multi-index during group operations
    group = group.reset_index()
    group['mfi'] = ind.calculate_mfi(group['price'], group['volume'])

    # Set index back to the original multi-index
    return group.set_index(['coin_id', 'date'])

# Apply the function within each 'coin_id' group
df = df.groupby('coin_id', observed=True, group_keys=False).apply(apply_mfi)

# Display the updated DataFrame with the MFI column
df.head()

In [ ]:
df.head(20)

In [ ]:
[importlib.reload(module) for module in modules]  # Reload all modules
config, metrics_config, modeling_config, experiments_config = u.load_all_configs('../config')  # Reload all configs

df2 = df[['ema_12','ema_26']].copy()

df2 = ind.add_crossover_column(df2, 'ema_12', 'ema_26', drop_col1=True, drop_col2=True)
df2.head()

In [ ]:
def identify_crossovers(series1, series2):
    """
    Identify crossovers between two time series.

    This function calculates the points where series1 crosses over series2.
    It handles NaN values by converting them to 0.

    Parameters:
    series1 (array-like): The first time series
    series2 (array-like): The second time series

    Returns:
    numpy.ndarray: An array of the same length as the input series, where:
        0 indicates no crossover
        1 indicates an upward crossover (series1 crosses above series2)
        -1 indicates a downward crossover (series1 crosses below series2)
    """
    diff = series1 - series2

    # Handle NaN values
    diff = np.nan_to_num(diff, nan=0.0)

    # Initialize crossovers array
    crossovers = np.zeros(len(series1))

    # Identify crossovers
    signs = np.sign(diff)
    sign_changes = signs[1:] != signs[:-1]
    crossover_indices = np.where(sign_changes)[0] + 1

    # Assign 1 for upward crossovers, -1 for downward crossovers
    crossovers[crossover_indices] = np.where(signs[crossover_indices] > 0, 1, -1)


In [ ]:
df[['ema_12','ema_26']]

In [ ]:

# Assuming `df` is your DataFrame with multi-index (coin_id, date) and ema_12, ema_26 columns

# Define a function that applies identify_crossovers to a group
def apply_crossovers(group):
    group['crossovers'] = identify_crossovers(group['ema_12'], group['ema_26'])
    return group

# Apply the function within each 'coin_id' group
df = df.groupby('coin_id', group_keys=False).apply(apply_crossovers)

# Display the resulting DataFrame with the new 'crossovers' column
df

In [ ]:
import pandas as pd



# Display the resulting DataFrame with the new 'crossovers' column
df

## Tests failing

In [ ]:
[importlib.reload(module) for module in modules]  # Reload all modules
config, metrics_config, modeling_config, experiments_config = u.load_all_configs('../config')  # Reload all configs



def sample_data():
    """
    Fixture to create a sample DataFrame for testing.

    Returns:
    - pd.DataFrame: A DataFrame with sample data for testing indicators.
    """
    return pd.DataFrame({
        'coin_id': ['BTC', 'BTC', 'BTC', 'ETH', 'ETH', 'ETH'],
        'date': pd.date_range(start='2023-01-01', periods=6),
        'price': [100, 110, 105, 200, 220, 210]
    })
sample_data=sample_data()

def sample_config():
    """
    Fixture to create a sample configuration for testing.

    Returns:
    - dict: A configuration dictionary for testing all supported indicators.
    """
    return {
        'time_series': {
            'market_data': {
                'price': {
                    'indicators': {
                        'sma': {'parameters': {'window': [2]}},
                        'ema': {'parameters': {'window': [2]}},
                        'rsi': {'parameters': {'window': [2]}},
                        'bollinger_bands_upper': {'parameters': {'window': [2], 'num_std': 2}},
                        'bollinger_bands_lower': {'parameters': {'window': [2], 'num_std': 2}}
                    }
                }
            }
        }
    }
sample_config=sample_config()
# @pytest.mark.unit
# def test_all_supported_indicators(sample_data, sample_config):
"""
Test that all supported indicators are correctly calculated and added to the DataFrame.

This test checks the calculation of SMA, EMA, RSI, and Bollinger Bands for the given sample data.
"""
result = ind.generate_time_series_indicators('market_data', sample_data, sample_config)

# Calculate expected values
# SMA (2-day)
# For BTC: [None, 105, 107.5]
# For ETH: [None, 210, 215]

# EMA (2-day)
# For BTC: [None, 106.67, 105.56]
# For ETH: [None, 213.33, 211.11]
# EMA = (Current * (2 / (1 + 2))) + (Previous EMA * (1 - (2 / (1 + 2))))

# RSI (2-day)
# For BTC: [None, 100, 33.33]
# For ETH: [None, 100, 33.33]
# RSI = 100 - (100 / (1 + (Average Gain / Average Loss)))

# Bollinger Bands (2-day, 2 std dev)
# Upper Band = SMA + (2 * std dev)
# Lower Band = SMA - (2 * std dev)
# For BTC: [None, 115, 112.5], [None, 95, 102.5]
# For ETH: [None, 230, 225], [None, 190, 205]

expected_columns = [
    'coin_id', 'date', 'price',
    'price_sma_2', 'price_ema_2', 'price_rsi_2',
    'price_bollinger_bands_upper_2', 'price_bollinger_bands_lower_2'
]

assert list(result.columns) == expected_columns

# Check SMA values
expected_sma = [np.nan, 105, 107.5, np.nan, 210, 215]
assert all(np.isclose(a, b, equal_nan=True) for a, b in zip(result['price_sma_2'], expected_sma))

# Check EMA values
expected_ema = [np.nan, 106.67, 105.56, np.nan, 213.33, 211.11]
assert all(np.isclose(a, b, equal_nan=True, rtol=1e-2) for a, b in zip(result['price_ema_2'], expected_ema))

# Check RSI values
expected_rsi = [np.nan, 1.0, 0.6667, np.nan, 1.0, 0.6667]
assert all(np.isclose(a, b, equal_nan=True, rtol=1e-2) for a, b in zip(result['price_rsi_2'], expected_rsi))

# Check Bollinger Bands values
expected_bb_upper = [np.nan, 115, 112.5, np.nan, 230, 225]
expected_bb_lower = [np.nan, 95, 102.5, np.nan, 190, 205]
assert all(np.isclose(a, b, equal_nan=True) for a, b in zip(result['price_bollinger_bands_upper_2'], expected_bb_upper))
assert all(np.isclose(a, b, equal_nan=True) for a, b in zip(result['price_bollinger_bands_lower_2'], expected_bb_lower))


In [ ]:
result['price_bollinger_bands_upper_2']

In [ ]:
import pandas as pd
import numpy as np

sample_data = pd.DataFrame({
    'price': [100, 110, 105, 200, 220, 210]
})

upper_band = ind.calculate_bollinger_bands(sample_data['price'], 'upper', window=2, num_std=2)
print(upper_band)

In [ ]:
expected_rsi

In [ ]:
result['price_rsi_2']

In [ ]:
expected_rsi

In [ ]:
pd.DataFrame(result['price_rsi_2'], expected_rsi)

In [ ]:
expected_columns

In [ ]:
result_df

In [ ]:
list(result_df.loc[result_df['coin_id'] == 'coin1', 'price_sma_2'].values) == list(expected_sma_2_coin1)

In [ ]:
list(expected_sma_2_coin1)

In [ ]:
result_mfi

In [ ]:
expected_rsi.values